In [28]:
%reset

In [29]:
import re
from collections import defaultdict
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 

In [30]:
no_topics=5
def fun():
    return [0 for i in range(no_topics)]

In [31]:
exp=r'[a-z]+'
no_topics=5
with open('tagmynews.txt',encoding='utf-8') as doc:
    txt=doc.readlines()
    feature_names=txt[0]
    newses=txt[1:]
newses=[" ".join(re.findall(exp,news.lower())) for news in newses] #conerting to lowercase and removing punctuations
# #print('######',feature_names)
exp='[a-zA-Z]+'
topics_words={}

with open(f'topic{str(no_topics)}.txt',encoding='utf-8') as doc:
    txt=doc.readlines()
    feature_names=[line.split(" ")[0] for line in txt]
# #print("feature names",feature_names) #unique no. of words

for line in txt:
    # #print(line)
    key=line.split(" ")[0]
    key=int(re.sub(exp,"",key))
    value=line.split(" ")[1:]
    topics_words[key]=value
# #print('topic_words',topics_words) #dictionary where topic is the key and values are words

counter=defaultdict(fun)
for news in range(len(newses)):
    # #print(news,newses[news])
    for word in newses[news].split(" "):
        if word!="" and word!=" ":
            for key,value in topics_words.items():
                if word in value:
                    counter[news][key]+=1
#print('counter',counter) #contains how many times a news contains words associated with each topics
keys=counter.keys() 
# #print('keys',keys,'len of keys',len(keys))

missing_values=[]
for idx in range(len(newses)):
    if idx not in keys:
        # #print(idx)
        missing_values.append(idx)

doc_topic_mat= pd.DataFrame.from_dict(counter).transpose()
doc_topic_mat.columns=feature_names
doc_topic_mat
doc_topic_mat.to_csv("count_matrix.csv")

probs=defaultdict(fun)
for row in doc_topic_mat.iterrows():
    # #print(row)
    total=row[1].sum()
    for val in range(len(row[1])):
        if total!=0:
            dist=row[1][val]/total
            probs[row[0]][val]=dist
        else:
            dist=row[1][val]
            # #print(row[0])
            probs[row[0]][val]=dist
final_dict=dict()
for key in sorted(probs.keys()):
    final_dict[key]=probs[key]
prob_dist=pd.DataFrame.from_dict(final_dict).transpose()
# prob_dist
# prob_dist.to_csv(f"gntm_topic_{no_topics}.csv")
#print("GNTM topic prob is generated")

In [32]:
prob_dist

,0,1,2,3,4
0,0.300000,0.200000,0.100000,0.100000,0.300000
1,0.461538,0.153846,0.076923,0.076923,0.230769
2,0.250000,0.250000,0.083333,0.166667,0.250000
3,0.285714,0.142857,0.142857,0.142857,0.285714
4,0.307692,0.230769,0.076923,0.153846,0.230769
...,...,...,...,...,...
20000,0.142857,0.214286,0.214286,0.214286,0.214286
20001,0.214286,0.285714,0.142857,0.142857,0.214286
20002,0.235294,0.235294,0.235294,0.117647,0.176471
20003,0.227273,0.181818,0.136364,0.181818,0.272727


In [41]:
len(topics_words)

5

In [48]:
pd.__version__

'1.5.0'

In [47]:
#print('######### Topics words ############ \n',topics_words)
df=[]
for key,value in  topics_words.items():
    df.append(" ".join(value))
df=pd.DataFrame(df,columns=['WORDS'])
print(df,'\n',df.shape)
cv = CountVectorizer()
x1 = cv.fit(df["WORDS"])
train=x1.transform(df["WORDS"])
y=list(df.index)

print("train shape",train.shape,"\n Train",train,y)
TOPIC_MODEL = MultinomialNB().fit(train,y)
print("Model Trained !")
test=x1.transform(newses) #changing all the newses in the dataset according to our count vectorizer
print("Test shape",test.shape)
Y_prob=TOPIC_MODEL.predict_proba(test)
bays_prob=pd.DataFrame(Y_prob)
bays_prob.drop(missing_values,axis=0,inplace=True)
bays_prob.to_csv(f"baysian_topic_{no_topics}.csv")
print("Bayesian topic prob is generated")

                                               WORDS
0   https politics trump news understand world co...
1   lady bird huffingtonpost movie fired https hi...
2   comey flynn huffingtonpost james director mic...
3   huffingtonpost https woman entertainment fast...
4   huffingtonpost https politics trump neutralit... 
 (5, 1)
train shape (5, 73) 
 Train   (0, 1)	1
  (0, 10)	1
  (0, 16)	1
  (0, 18)	1
  (0, 28)	1
  (0, 30)	1
  (0, 31)	1
  (0, 32)	1
  (0, 40)	1
  (0, 48)	1
  (0, 49)	1
  (0, 51)	1
  (0, 53)	1
  (0, 54)	1
  (0, 60)	1
  (0, 63)	1
  (0, 65)	1
  (0, 66)	1
  (0, 67)	1
  (0, 72)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 11)	1
  (1, 13)	1
  :	:
  (3, 62)	1
  (3, 63)	1
  (3, 65)	1
  (3, 69)	1
  (3, 71)	1
  (4, 2)	1
  (4, 7)	1
  (4, 12)	1
  (4, 13)	1
  (4, 14)	1
  (4, 23)	1
  (4, 28)	1
  (4, 29)	1
  (4, 30)	1
  (4, 31)	1
  (4, 35)	1
  (4, 44)	1
  (4, 46)	1
  (4, 47)	1
  (4, 48)	1
  (4, 50)	1
  (4, 53)	1
  (4, 57)	1
  (4, 63)	1
  (4, 70)	1 [0, 1, 2, 3, 4]

 Smoothed feature count 
 [[

In [34]:
pd.DataFrame.sparse.from_spmatrix(test)
# test

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [35]:
Y_prob=TOPIC_MODEL.predict_proba(test[32])
#print(Y_prob)

before check_x  (1, 73)
Inside check X function !!!!
after check X (1, 73)
########## Inside joint liklyhood function ################
########## Dot product ########## [[-11.51835694 -12.21150412 -12.9046513  -12.9046513  -12.21150412]] ####
 shape of X (1, 73) Shape of feature log probability (5, 73)
###################################

 X is   (0, 28)	1
  (0, 30)	1
  (0, 72)	1

 feature log prob is [[-4.53259949 -3.83945231 -4.53259949 -4.53259949 -4.53259949 -4.53259949
  -4.53259949 -4.53259949 -4.53259949 -4.53259949 -3.83945231 -4.53259949
  -4.53259949 -4.53259949 -4.53259949 -4.53259949 -3.83945231 -4.53259949
  -3.83945231 -4.53259949 -4.53259949 -4.53259949 -4.53259949 -4.53259949
  -4.53259949 -4.53259949 -4.53259949 -4.53259949 -3.83945231 -4.53259949
  -3.83945231 -3.83945231 -3.83945231 -4.53259949 -4.53259949 -4.53259949
  -4.53259949 -4.53259949 -4.53259949 -4.53259949 -3.83945231 -4.53259949
  -4.53259949 -4.53259949 -4.53259949 -4.53259949 -4.53259949 -4.53259949
  -

In [36]:
ex=dict()
for key,values in x1.vocabulary_.items():
    ex[values]=key
feature_names=[]
for i in sorted(ex.keys()):
    # #print(i,ex[i])
    feature_names.append(ex[i])

In [37]:
feature_names

['amatulli',
 'andy',
 'announces',
 'best',
 'bird',
 'boboltz',
 'christmas',
 'comedy',
 'comey',
 'conyers',
 'cook',
 'decade',
 'democrat',
 'destroys',
 'dicker',
 'director',
 'donald',
 'elyse',
 'entertainment',
 'evidence',
 'fast',
 'fired',
 'flynn',
 'good',
 'guilty',
 'handle',
 'history',
 'house',
 'https',
 'huffingtonpost',
 'huffpost',
 'incredible',
 'issues',
 'james',
 'jenna',
 'jordan',
 'know',
 'korea',
 'lady',
 'london',
 'mary',
 'mcdonald',
 'michael',
 'missile',
 'moran',
 'movie',
 'need',
 'neutrality',
 'news',
 'north',
 'order',
 'papenfuss',
 'people',
 'politics',
 'president',
 'project',
 'protest',
 'ryan',
 'science',
 'shots',
 'shows',
 'single',
 'street',
 'trump',
 'tweeted',
 'understand',
 'video',
 'voices',
 'wanshel',
 'woman',
 'women',
 'wonder',
 'world']

In [38]:
train.columns=feature_names